In [ ]:
%run -i 'cosim_import.py'

`caseH` is the start time [H] of the co-simulation, now 10 and 18 are supported.

In [ ]:
caseH = 18
ict = True
ecc = True
agc = True

%run -i 'cosim_setup_andes.py'

## Load synthetic

Create load data ``d_syn``, which is a ``DataFrame`` that have three columns: ``time``, ``s10``, ``h10``. ``time`` is by seconds, ``s10`` is scalar load.

In [ ]:
%run -i 'cosim_loadsyn.py'

## Setup ``ssp`` and ``ssd``

Convert ``ssa`` to pandapower net ``ssp``, add generator cost

The input cost array follow the matpower/pypower format, now only poly_cost is supported

In [ ]:
%run -i 'cosim_setup_dispatch.py'
dc_comp.round(4)

## Prepare

### Make link table

In [ ]:
%run -i 'cosim_linktable.py'
ssa_key2

## Loop

In the loop, there are mainly X parts:

- interval RTED: run DCOPF (``ssd.mdl``), run ACOPF(``ssp``), 

- interval AGC: do AGC, do dispatch with smooth setpoints

- interval PQ: alter load, run TDS(``ssa.TDS``)

Notes:

- The setpoints for DG are coded but not verified yet.

- After the development of ANDES control room, the dispatch and AGC part can be refactored.

- interface variables: ``DG.pmx``: DPV profile, ``DG.pref0``: setpoints, ``DG.pext0``: AGC

- ACOPF in pandapower considered generator limtis of ramping and SFR reserve

In [ ]:
# Change ``rru``, ``rrd``, ``rsfr``, ``t_total`` for necessary.
t_total = 800

rru, rrd, rsfr = 1.0, 1.0, 0.05

# Define functions and constants used in loop
%run -i 'cosim_const.py'

In [ ]:
%run -i 'cosim_main.py'

In [ ]:
# sse.tsd.iloc[30:50][['Pet', 'Ptc', 'Pt']]

In [ ]:
sse.ict()
sse.ev['ict'].hist()
sse.ev['tt'].hist()

In [ ]:
sse.plot_agc(style='ieee')

In [ ]:
%run -i 'cosim_plot.py'
ax_gen[0, 1].set_ylim([-40, 60])